# Черновой ноутбук с базовым решением

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 100)

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

os.environ['MKL_NUM_THREADS'] = '1'

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
import src.features as ft
from src.recommenders import MainRecommender

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('./data/retail_train.csv')
item_features = pd.read_csv('./data/product.csv')
user_features = pd.read_csv('./data/hh_demographic.csv')

In [3]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'

# N = Neighbors
N_PREDICT = 10 

## Подготовка фичей датасета

In [4]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

## Разбиваю датасет на train, eval, test

In [5]:
VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

In [6]:
# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]


# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [7]:
# сделаем объединенный сет данных для первого уровня (матчинга)
df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])

In [8]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")

In [9]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (2108779, 12) Users: 2498 Items: 83685
val_matcher
Shape: (169711, 12) Users: 2154 Items: 27649
train_ranker
Shape: (169711, 12) Users: 2154 Items: 27649
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329


In [10]:
data_train_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


## Предфильтрация товаров

In [11]:
# Определю количество наиболее популярных товаров, которые составляют 90% продаж
items_by_poprularity = data_train_matcher.groupby(by='item_id')['basket_id'].nunique(). \
    reset_index().sort_values(by='basket_id', ascending=False)
items_by_poprularity.rename(columns={'basket_id': 'n_purchases'}, inplace=True)
items_by_poprularity.head()

,item_id,n_purchases
34192,1082185,24318
54389,6534178,16233
28450,1029743,11661
24657,995242,10226
36790,1106523,8011


In [12]:
purchses_sum = items_by_poprularity.n_purchases.sum()
top_90_percent_items_list = []
purchses_commul_sum = 0
for item, n_purchases in zip(items_by_poprularity.item_id, items_by_poprularity.n_purchases):
    purchses_commul_sum += n_purchases
    if (purchses_commul_sum / purchses_sum) < 0.9:
        top_90_percent_items_list.append(item)
    else:
        break

n_popular = len(top_90_percent_items_list)
print(f'90% top items: {n_popular}')

90% top items: 18714


In [13]:
# Оставляю в датасете только 18714 товаров. Id остальных заменю на 999999
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, 
                                     take_n_popular=n_popular)

n_items_after = data_train_matcher['item_id'].nunique()
print(f'Decreased # items from {n_items_before} to {n_items_after}')

Decreased # items from 83685 to 18715


## Убираю холодный пользователей

In [14]:
# ищем общих пользователей
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (861404, 13) Users: 2495 Items: 18715
val_matcher
Shape: (169615, 12) Users: 2151 Items: 27644
train_ranker
Shape: (169615, 12) Users: 2151 Items: 27644
val_ranker
Shape: (118282, 12) Users: 2040 Items: 24325


GRID SEARCH

In [15]:
# factors = [20, 30, 40,]
# regularization = [0.1, 0.01, 0.001]
# iterations = [10, 15, 20]

In [16]:
# from itertools import product

In [17]:
# res_dict = {
#     'recall': [],
#     'precision': [],
#     'factors': [], 
#     'regularization': [], 
#     'iterations': []
# }

In [18]:
# %%time
# res_list = []
# for factor, reg, iteration in product(factors, regularization, iterations):
#     recommender_params = {'factors': factor, 'regularization': reg, 'iterations': iteration, 
#                           'num_threads': 6, 'random_state': 0}
#     base_recommender = MainRecommender(data_train_matcher, weighting='tfidf',
#                              model_type='als', own_recommender_type='cosine', 
#                              user_item_matrix_values='binary', recommender_params=recommender_params)
#     recall = base_recommender.evalMetrics(metric_type='recall', df_result=data_val_matcher, 
#                     target_col_name=USER_COL, recommend_model_type='rec', N_PREDICT=10)
#     precision = base_recommender.evalMetrics(metric_type='precision', df_result=data_val_matcher, 
#                     target_col_name=USER_COL, recommend_model_type='rec', N_PREDICT=5)
#     res_list.append({'recall': recall, 'precision': precision, 'factors': factor, 
#                      'regularization': reg, 'iterations': iteration})

In [19]:
# res_df = pd.DataFrame(res_list)
# res_df.head()

In [20]:
# for col in res_df.columns.to_list()[:2]:
#     print(f'Best {col}:\n{res_df.loc[np.argmax(res_df[col]), :]}')
#     print('*' * 30)

# Обучаю рекомендательную модель

In [21]:
recommender_params = {'factors': 40, 'regularization': 0.001, 'iterations': 20, 
                          'num_threads': 6, 'random_state': 0}

In [22]:
recommender = MainRecommender(data_train_matcher, weighting='tfidf',
                             model_type='als', own_recommender_type='cosine', 
                             user_item_matrix_values='binary', recommender_params=recommender_params)

### Recall@10 of matching

In [23]:
recs_type_list = ['own', 'rec', 'itm', 'usr']

In [24]:
TOPK_RECALL = 10

In [25]:
for el in recs_type_list:
    res = recommender.evalMetrics(metric_type='recall', df_result=data_val_matcher, 
                    target_col_name=USER_COL, recommend_model_type=el, N_PREDICT=TOPK_RECALL)
    print(f'{el} recall: {res}')

own recall: 0.05173337509903519
rec recall: 0.02601904830492872
itm recall: 0.011959812042702501
usr recall: 0.013391522923271991


Для дальнейшего ранжирования буду использовать own_recommender, т.к. он показал наибольшее значение $recall@k$

# Ранжирование

## Подготовка данных для трейна

In [26]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [27]:
# собираем кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(
    lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

In [28]:
df_match_candidates.head(2)

,user_id,candidates
0,2070,"[1029743, 913210, 1105426, 933067, 838186, 109..."
1,2021,"[1119454, 950935, 1041390, 844179, 1013928, 65..."


In [29]:
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1
                                    ).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

In [30]:
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [31]:
df_match_candidates.head(4)

,user_id,item_id
0,2070,1029743
0,2070,913210
0,2070,1105426
0,2070,933067


### Проверка "теплого" старта

In [32]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates
Shape: (21510, 2) Users: 2151 Items: 6322


### Создаю трейн сет для ранжирования с учетом кандидатов с этапа 1 

In [33]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 

In [34]:
df_ranker_train.head()

,user_id,item_id,target
2104867,2070,1019940,1
2107468,2021,840361,1
2107469,2021,856060,1
2107470,2021,869344,1
2107471,2021,896862,1


#### Не хватает нулей в датасете, поэтому добавляем наших кандитатов в качество нулей

In [35]:
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

df_ranker_train['target'].fillna(0, inplace= True)

In [36]:
df_ranker_train.target.value_counts()

0.0    15950
1.0     5556
Name: target, dtype: int64

In [37]:
df_ranker_train.head(2)

,user_id,item_id,target
0,2070,1029743,0.0
1,2070,913210,1.0


## Подготавливаем фичи для обучения модели

In [38]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [39]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [40]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1029743,0.0,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,913210,1.0,2,GROCERY,National,WATER - CARBONATED/FLVRD DRINK,NON-CRBNTD DRNKING/MNERAL WATE,405.6 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


### Добавляю новые фичи в датасет
Заказ товара в последних 5 транзакциях в виде последовательности бит (категориальная).

In [41]:
last_5_purchases_as_bit_array = ft.get_last_5_purchases_as_bit_array(data_train_ranker, item_features)
df_ranker_train = df_ranker_train.merge(last_5_purchases_as_bit_array, on=['item_id'], how='left')

#### user_id


In [42]:
# Средняя сумма покупки 1 товара в каждой категории
mean_purchase_by_department= ft.get_mean_purchase_per_item_by_department(data_train_ranker, 
                                                                      item_features)
df_ranker_train = df_ranker_train.merge(mean_purchase_by_department[['user_id', 
                        'department', 'mean_purchase']], on=['user_id', 'department'], how='left')

# Кол-во покупок в каждой категории
num_purchases_by_department = ft.get_num_purchases_per_department(data_train_ranker, item_features)
df_ranker_train = df_ranker_train.merge(num_purchases_by_department[['user_id', 'department', 
                                        'num_purchases']], on=['user_id', 'department'], how='left')

# Доля покупок утром/днем/вечером
user_trans_df = ft.get_proportion_of_purchases_by_times_of_day(data_train_ranker)
df_ranker_train = df_ranker_train.merge(user_trans_df, on='user_id', how='left')

# Кол-во магазинов, в которых продавался товар
n_stores = ft.get_n_stores(data_train_ranker)
df_ranker_train = df_ranker_train.merge(n_stores, on=['item_id'], how='left')

#### item_id

In [43]:
# Кол-во покупок в неделю
week_purchases_df = ft.get_num_purchases_per_week(data_train_ranker)
df_ranker_train = df_ranker_train.merge(week_purchases_df[['item_id', 'n_purchases_per_week']], 
                                        on=['item_id'], how='left')

# Среднее кол-во покупок 1 товара в категории в неделю
mean_n_purchases_per_week = ft.get_mean_num_purchases_per_item_dept_week(data_train_ranker, 
                                                                      item_features)
df_ranker_train = df_ranker_train.merge(mean_n_purchases_per_week[['department', 
                                        'mean_n_purchases_per_week']], on=['department'], how='left')

# (Кол-во покупок в неделю) / (Среднее кол-во покупок 1 товара в категории в неделю)
df_ranker_train['n_purchases_div_by_mean'] = \
    df_ranker_train.n_purchases_per_week / df_ranker_train.mean_n_purchases_per_week

# Цена
item_price_df = ft.get_price(data_train_ranker)
df_ranker_train = df_ranker_train.merge(item_price_df[['item_id', 'price']], 
                                        on=['item_id'], how='left')

# Цена / Средняя цена товара в категории
df_ranker_train.loc[(df_ranker_train.department == 'PRODUCE') &
                    (df_ranker_train.price == float('Inf')), 'price'] = 0
mean_price_by_department = ft.get_mean_price_by_department(df_ranker_train)
df_ranker_train = df_ranker_train.merge(mean_price_by_department[['department', 'mean_price']], 
                                        on=['department'], how='left')
df_ranker_train['price_div_by_mean_dept_price'] = df_ranker_train.price / df_ranker_train.mean_price
df_ranker_train.drop('mean_price', axis=1, inplace=True)

# Кол-во транзакций клиента
n_transactions = ft.get_n_transactions(data_train_ranker)
df_ranker_train = df_ranker_train.merge(n_transactions, on=['user_id'], how='left')

# mean / max / std кол-ва уникальных товаров в корзине клиента
unique_items = ft.get_unique_items_in_basket(data_train_ranker)
df_ranker_train = df_ranker_train.merge(unique_items, on=['user_id'], how='left')

# mean / max / std кол-ва уникальных категорий в корзине клиента
nunique_departments = ft.get_unique_departments_in_basket(data_train_ranker, item_features)
df_ranker_train = df_ranker_train.merge(nunique_departments, on=['user_id'], how='left')

#### user_id - item_id

In [44]:
# (Кол-во покупок юзером конкретной категории в неделю) - (Среднее 
# кол-во покупок всеми юзерами конкретной категории в неделю)
n_purchases_sub_by_mean = ft.get_num_purchases_sub_by_mean(data_train_ranker, item_features)
df_ranker_train = df_ranker_train.merge(n_purchases_sub_by_mean[['user_id', 'department',
        'n_purchases_sub_by_mean']], on=['user_id', 'department'], how='left')

# (Кол-во покупок юзером конкретной категории в неделю) / (Среднее 
# кол-во покупок всеми юзерами конкретной категории в неделю)
n_purchases_div_by_mean = ft.get_num_purchases_div_by_mean_all_users(data_train_ranker, item_features)
df_ranker_train = df_ranker_train.merge(n_purchases_div_by_mean[['user_id', 'department',
        'n_purchases_div_by_mean_all_users']], on=['user_id', 'department'], how='left')

# (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id)
sales_values_by_dept = ft.get_mean_sales_value_per_item_by_department(data_train_ranker, item_features)
df_ranker_train = df_ranker_train.merge(sales_values_by_dept[['department', 'mean_sale_sum_per_item']], 
                                        on=['department'], how='left')
df_ranker_train['mean_sale_sum_per_item_sub_price'] = \
    df_ranker_train.mean_sale_sum_per_item - df_ranker_train.price
df_ranker_train.drop('mean_sale_sum_per_item', axis=1, inplace=True)

Поведенческие фичи

In [45]:
# Общая сумма покупок каждого товара
total_item_sales_value = ft.get_total_item_sales_value(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(total_item_sales_value, how='left', on='item_id')

# Общее количество по каждому товару
total_quantity_value = ft.get_total_quantity_value(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(total_quantity_value, how='left', on='item_id')

# Количество покупателей по каждому товару
item_freq = ft.get_item_freq(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(item_freq, how='left', on='item_id')

# Частота пользователей
user_freq = ft.get_user_freq(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(user_freq, how='left', on=USER_COL)

# Общее количество покупок по каждому пользователю
total_user_sales_value = ft.get_total_user_sales_value(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(total_user_sales_value, how='left', on='user_id')

# Среднее количество покупок товара в неделю
item_quantity_per_week = ft.get_item_quantity_per_week(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(item_quantity_per_week, how='left', on='item_id')

# Среднее количество купленного товара пользователем в неделю
user_quantity_per_week = ft.get_user_quantity_per_week(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(user_quantity_per_week, how='left', on='user_id')

# Среднее количество товара за 1 покупку
item_quantity_per_basket = ft.get_item_quantity_per_basket(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(item_quantity_per_basket, how='left', on='item_id')

# Среднее количество товара у польователя за 1 покупку
user_quantity_per_basket = ft.get_user_quantity_per_basket(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(user_quantity_per_basket, how='left', on='user_id')

# Средняя частота товара в карзине
item_freq_per_basket = ft.get_item_freq_per_basket(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(item_freq_per_basket, how='left', on='item_id')

# Средняя частота пользователей купивших товар
user_freq_per_basket = ft.get_user_freq_per_basket(df_join_train_matcher)
df_ranker_train = df_ranker_train.merge(user_freq_per_basket, how='left', on='user_id')

Факторы товаров из модели матричной факторизации

In [46]:
item_factors = recommender.get_item_factors()
df_ranker_train = df_ranker_train.merge(item_factors, on=['item_id'], how='left')

Факторы пользователей из модели матричной факторизации

In [47]:
user_factors = recommender.get_user_factors()
df_ranker_train = df_ranker_train.merge(user_factors, on=['user_id'], how='left')

### Разбиваю на X и y

In [48]:
df_ranker_train.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,item_in_last_5_transactions,mean_purchase,num_purchases,morning_trans,day_trans,evening_trans,n_stores,n_purchases_per_week,mean_n_purchases_per_week,n_purchases_div_by_mean,price,price_div_by_mean_dept_price,n_transactions,mean_unique_items,max_unique_items,std_unique_items,mean_unique_departments,max_unique_departments,std_unique_departments,n_purchases_sub_by_mean,n_purchases_div_by_mean_all_users,mean_sale_sum_per_item_sub_price,total_item_sales_value,total_quantity_value,item_freq,user_freq,total_user_sales_value,item_quantity_per_week,user_quantity_per_week,item_quantity_per_basket,user_quantity_per_basket,item_freq_per_basket,user_freq_per_basket,item_factor_0,...,item_factor_30,item_factor_31,item_factor_32,item_factor_33,item_factor_34,item_factor_35,item_factor_36,item_factor_37,item_factor_38,item_factor_39,user_factor_0,user_factor_1,user_factor_2,user_factor_3,user_factor_4,user_factor_5,user_factor_6,user_factor_7,user_factor_8,user_factor_9,user_factor_10,user_factor_11,user_factor_12,user_factor_13,user_factor_14,user_factor_15,user_factor_16,user_factor_17,user_factor_18,user_factor_19,user_factor_20,user_factor_21,user_factor_22,user_factor_23,user_factor_24,user_factor_25,user_factor_26,user_factor_27,user_factor_28,user_factor_29,user_factor_30,user_factor_31,user_factor_32,user_factor_33,user_factor_34,user_factor_35,user_factor_36,user_factor_37,user_factor_38,user_factor_39
0,2070,1029743,0.0,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,00000,1.461549,39.0,0.714286,0.0,0.285714,110.0,179.333333,2.499367,71.751510,2.682058,0.655522,43,4.744186,50,10.309711,1.790698,8,1.319418,6.192347,21.127701,-0.150089,35764.66,15015,12737,1996,5754.86,165.000000,1218.32967,0.061233,0.452137,0.051943,0.00814,0.037637,...,-0.000454,0.048695,0.046490,0.010612,-0.025334,0.075626,0.019065,-0.020496,-0.003342,0.054763,1.422094,2.219372,-2.770137,0.800567,-0.804039,1.347933,4.091575,2.017889,-0.854053,-1.050599,-3.817773,-1.70245,-0.40162,1.334348,2.105724,1.782048,2.917558,-0.999815,0.656557,-3.221031,-1.855689,0.635844,1.332974,-1.305028,2.994377,0.960438,-0.327382,1.116586,-0.24491,-0.816451,0.543538,-0.793592,0.509429,0.516272,-0.012325,2.121768,1.396681,-1.942739,3.767898,2.183004
1,2070,913210,1.0,2,GROCERY,National,WATER - CARBONATED/FLVRD DRINK,NON-CRBNTD DRNKING/MNERAL WATE,405.6 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,00000,1.461549,39.0,0.714286,0.0,0.285714,54.0,17.666667,2.499367,7.068457,3.990000,0.975196,43,4.744186,50,10.309711,1.790698,8,1.319418,6.192347,21.127701,-1.458031,5406.18,1364,1175,1996,5754.86,14.989011,1218.32967,0.005563,0.452137,0.004792,0.00814,0.075757,...,-0.014227,-0.004915,-0.013485,0.038461,0.019118,0.018813,0.019707,0.009978,0.012164,0.056061,1.422094,2.219372,-2.770137,0.800567,-0.804039,1.347933,4.091575,2.017889,-0.854053,-1.050599,-3.817773,-1.70245,-0.40162,1.334348,2.105724,1.782048,2.917558,-0.999815,0.656557,-3.221031,-1.855689,0.635844,1.332974,-1.305028,2.994377,0.960438,-0.327382,1.116586,-0.24491,-0.816451,0.543538,-0.793592,0.509429,0.516272,-0.012325,2.121768,1.396681,-1.942739,3.767898,2.183004
2,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,00000,3.596667,2.0,0.714286,0.0,0.285714,1.0,1.000000,2.260583,0.442364,3.990000,0.861000,43,4.744186,50,10.309711,1.790698,8,1.319418,0.808016,5.208760,0.537463,442.90,113,99,1996,5754.86,1.241758,1218.32967,0.000461,0.452137,0.000404,0.00814,0.036260,...,0.032958,0.001080,-0.009906,0.005463,-0.004724,0.014826,0.012331,0.006038,0.002804,0.001886,1.422094,2.219372,-2.770137,0.800567,-0.804039,1.347933,4.091575,2.017889,-0.854053,-1.050599,-3.817773,-1.70245,-0.40162,1.334348,2.105724,1

In [49]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [50]:
cat_feats = X_train.columns[2:16].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

## Обучение модели ранжирования

GRID SEARCH

In [57]:
max_depth_list = [5, 10, 15, 20, 30]
n_estimators_list = [400, 500, 600, 700]
learning_rate_list = [0.1, 0.05, 0.01]
n_iters = len(max_depth_list) * len(n_estimators_list) * len(learning_rate_list)

In [52]:
from itertools import product

In [59]:
%%time
i = 0
res_list = []
for max_depth, n_estimators, learning_rate in product(max_depth_list, 
                                                      n_estimators_list, learning_rate_list):
    i += 1
    base_lgb = LGBMClassifier(objective='binary',
                             max_depth=max_depth,
                             n_estimators=n_estimators,
                             learning_rate=learning_rate,
                             categorical_column=cat_feats,
                             random_state=0)
    base_lgb.fit(X_train, y_train)
    base_train_preds = base_lgb.predict_proba(X_train)
    base_df_ranker_predict = df_ranker_train.copy()
    base_df_ranker_predict['proba_item_purchase'] = base_train_preds[:,1]
    ranker_precision = recommender.evalMetrics(metric_type='precision', df_result=data_val_ranker, 
                    target_col_name=USER_COL, recommend_model_type='own', N_PREDICT=5)
    reranked_precision = recommender.reranked_metrics(metric_type='precision', 
                            df_result=data_val_ranker, df_predict=base_df_ranker_predict,
                            target_col_name=USER_COL, recommend_model_type='own', N_PREDICT=5)
    res_list.append({'ranker_precision': ranker_precision, 'reranked_precision': reranked_precision,
                    'max_depth': max_depth, 'n_estimators': n_estimators,
                    'learning_rate': learning_rate})
    print(f'{i}/{n_iters}: ranker_precision: {ranker_precision}, reranked_precision: {reranked_precision}, max_depth: {max_depth}, n_estimators: {n_estimators}, learning_rate: {learning_rate}')

1/60: ranker_precision: 0.2759803921568615, reranked_precision: 0.30558746736292336, max_depth: 5, n_estimators: 400, learning_rate: 0.1
2/60: ranker_precision: 0.2759803921568615, reranked_precision: 0.30516971279373306, max_depth: 5, n_estimators: 400, learning_rate: 0.05
3/60: ranker_precision: 0.2759803921568615, reranked_precision: 0.28710182767623876, max_depth: 5, n_estimators: 400, learning_rate: 0.01
4/60: ranker_precision: 0.2759803921568615, reranked_precision: 0.305169712793733, max_depth: 5, n_estimators: 500, learning_rate: 0.1
5/60: ranker_precision: 0.2759803921568615, reranked_precision: 0.30631853785900737, max_depth: 5, n_estimators: 500, learning_rate: 0.05
6/60: ranker_precision: 0.2759803921568615, reranked_precision: 0.2885639686684059, max_depth: 5, n_estimators: 500, learning_rate: 0.01
7/60: ranker_precision: 0.2759803921568615, reranked_precision: 0.3054830287206261, max_depth: 5, n_estimators: 600, learning_rate: 0.1
8/60: ranker_precision: 0.275980392156861

In [64]:
res_df = pd.DataFrame(res_list)
res_df.iloc[np.argmax(res_df['reranked_precision']), :]

ranker_precision        0.275980
reranked_precision      0.308825
max_depth              10.000000
n_estimators          700.000000
learning_rate           0.050000
Name: 22, dtype: float64

In [78]:
lgb = LGBMClassifier(objective='binary',
                     max_depth=10,
                     n_estimators=700,
                     learning_rate=0.05,
                     categorical_column=cat_feats,
                     random_state=0)

In [79]:
lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

In [80]:
df_ranker_predict = df_ranker_train.copy()

In [81]:
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

In [82]:
df_ranker_predict.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,item_in_last_5_transactions,mean_purchase,num_purchases,morning_trans,day_trans,evening_trans,n_stores,n_purchases_per_week,mean_n_purchases_per_week,n_purchases_div_by_mean,price,price_div_by_mean_dept_price,n_transactions,mean_unique_items,max_unique_items,std_unique_items,mean_unique_departments,max_unique_departments,std_unique_departments,n_purchases_sub_by_mean,n_purchases_div_by_mean_all_users,mean_sale_sum_per_item_sub_price,total_item_sales_value,total_quantity_value,item_freq,user_freq,total_user_sales_value,item_quantity_per_week,user_quantity_per_week,item_quantity_per_basket,user_quantity_per_basket,item_freq_per_basket,user_freq_per_basket,item_factor_0,...,item_factor_31,item_factor_32,item_factor_33,item_factor_34,item_factor_35,item_factor_36,item_factor_37,item_factor_38,item_factor_39,user_factor_0,user_factor_1,user_factor_2,user_factor_3,user_factor_4,user_factor_5,user_factor_6,user_factor_7,user_factor_8,user_factor_9,user_factor_10,user_factor_11,user_factor_12,user_factor_13,user_factor_14,user_factor_15,user_factor_16,user_factor_17,user_factor_18,user_factor_19,user_factor_20,user_factor_21,user_factor_22,user_factor_23,user_factor_24,user_factor_25,user_factor_26,user_factor_27,user_factor_28,user_factor_29,user_factor_30,user_factor_31,user_factor_32,user_factor_33,user_factor_34,user_factor_35,user_factor_36,user_factor_37,user_factor_38,user_factor_39,proba_item_purchase
0,2070,1029743,0.0,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,00000,1.461549,39.0,0.714286,0.0,0.285714,110.0,179.333333,2.499367,71.751510,2.682058,0.655522,43,4.744186,50,10.309711,1.790698,8,1.319418,6.192347,21.127701,-0.150089,35764.66,15015,12737,1996,5754.86,165.000000,1218.32967,0.061233,0.452137,0.051943,0.00814,0.037637,...,0.048695,0.046490,0.010612,-0.025334,0.075626,0.019065,-0.020496,-0.003342,0.054763,1.422094,2.219372,-2.770137,0.800567,-0.804039,1.347933,4.091575,2.017889,-0.854053,-1.050599,-3.817773,-1.70245,-0.40162,1.334348,2.105724,1.782048,2.917558,-0.999815,0.656557,-3.221031,-1.855689,0.635844,1.332974,-1.305028,2.994377,0.960438,-0.327382,1.116586,-0.24491,-0.816451,0.543538,-0.793592,0.509429,0.516272,-0.012325,2.121768,1.396681,-1.942739,3.767898,2.183004,0.654429
1,2070,913210,1.0,2,GROCERY,National,WATER - CARBONATED/FLVRD DRINK,NON-CRBNTD DRNKING/MNERAL WATE,405.6 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,00000,1.461549,39.0,0.714286,0.0,0.285714,54.0,17.666667,2.499367,7.068457,3.990000,0.975196,43,4.744186,50,10.309711,1.790698,8,1.319418,6.192347,21.127701,-1.458031,5406.18,1364,1175,1996,5754.86,14.989011,1218.32967,0.005563,0.452137,0.004792,0.00814,0.075757,...,-0.004915,-0.013485,0.038461,0.019118,0.018813,0.019707,0.009978,0.012164,0.056061,1.422094,2.219372,-2.770137,0.800567,-0.804039,1.347933,4.091575,2.017889,-0.854053,-1.050599,-3.817773,-1.70245,-0.40162,1.334348,2.105724,1.782048,2.917558,-0.999815,0.656557,-3.221031,-1.855689,0.635844,1.332974,-1.305028,2.994377,0.960438,-0.327382,1.116586,-0.24491,-0.816451,0.543538,-0.793592,0.509429,0.516272,-0.012325,2.121768,1.396681,-1.942739,3.767898,2.183004,0.840897
2,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,00000,3.596667,2.0,0.714286,0.0,0.285714,1.0,1.000000,2.260583,0.442364,3.990000,0.861000,43,4.744186,50,10.309711,1.790698,8,1.319418,0.808016,5.208760,0.537463,442.90,113,99,1996,5754.86,1.241758,1218.32967,0.000461,0.452137,0.000404,0.00814,0.036260,...,0.001080,-0.009906,0.005463,-0.004724,0.014826,0.012331,0.006038,0.002804,0.001886,1.422094,2.219372,-2.770137,0.800567,-0.804039,1.347933,4.091575,2.017889,-0.854053,-1.050599,-3.817773,-1.70245,-0.40162,1.334348,2.105724,1.78204

# Evaluation on test dataset

In [83]:
TOPK_PRECISION = 5

In [84]:
print('Matcher\'s precision:')
for el in recs_type_list:
    res = recommender.evalMetrics(metric_type='precision', df_result=data_val_matcher, 
                    target_col_name=USER_COL, recommend_model_type=el, N_PREDICT=TOPK_PRECISION)
    print(f'{el} precision: {res}')

Matcher's precision:
own precision: 0.3266387726638777
rec precision: 0.13463505346350355
itm precision: 0.06824732682473293
usr precision: 0.0733612273361227


In [85]:
print('Ranker\'s precision:')
for el in recs_type_list:
    res = recommender.evalMetrics(metric_type='precision', df_result=data_val_ranker, 
                    target_col_name=USER_COL, recommend_model_type=el, N_PREDICT=TOPK_PRECISION)
    print(f'{el} precision: {res}')

Ranker's precision:
own precision: 0.2759803921568615
rec precision: 0.107450980392156
itm precision: 0.05813725490196123
usr precision: 0.05970588235294166


## Eval re-ranked matched result on test dataset

In [86]:
print('Re-ranked precision:')
recommender.reranked_metrics(metric_type='precision', df_result=data_val_ranker, 
                                  df_predict=df_ranker_predict, target_col_name=USER_COL, 
                                  recommend_model_type='own', N_PREDICT=TOPK_PRECISION)

Re-ranked precision:


0.3088250652741511

# Оценка на тесте для выполнения курсового проекта

In [87]:
df_test = pd.read_csv('./data/retail_test1.csv')
df_test.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0
3,1602,41665647035,664,827939,1,7.99,334,0.0,1741,96,0.0,0.0
4,1602,41665647035,664,927712,1,0.59,334,-0.4,1741,96,0.0,0.0


In [88]:
# теплый старт
df_test = df_test[df_test.user_id.isin(common_users)]
print_stats_data(df_test,'df_test')

df_test
Shape: (88665, 12) Users: 1883 Items: 20492


In [89]:
print('Test precision:')
for el in recs_type_list:
    res = recommender.evalMetrics(metric_type='precision', df_result=df_test, 
                target_col_name=USER_COL, recommend_model_type=el, N_PREDICT=TOPK_PRECISION)
    print(f'{el} precision: {res}')

Test precision:
own precision: 0.23027084439723633
rec precision: 0.09134360063728064
itm precision: 0.05246946362188031
usr precision: 0.045990440785980136


In [90]:
print('Test re-ranked precision:')
recommender.reranked_metrics(metric_type='precision', df_result=df_test, 
                                  df_predict=df_ranker_predict, target_col_name=USER_COL, 
                                  recommend_model_type='own', N_PREDICT=TOPK_PRECISION)

Test re-ranked precision:


0.25135135135134906